In [53]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import logging 
import yaml

In [54]:
with open("../cofig.yaml" , "r") as f:
    config = yaml.full_load(f)

In [ ]:
# Configure logging
logging.basicConfig(
    filename="Data-Ingestion-To-Sql.log",
    level=logging.INFO,   
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [56]:
df = pd.read_csv(config['file_path']['raw_file'])

logging.info("Data lod")

In [57]:
df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [58]:
String_hostname =  config['Database']['String_hostname']
String_database = config['Database']['String_database']
String_port = config['Database']['String_port']
String_username = config['Database']['String_username']
String_password = config['Database']['String_password']

In [59]:
# pip install pymysql

In [65]:
from logging import exception

try:
    engine = create_engine( f"mysql+pymysql://{String_username}:{String_password}@{String_hostname}:{String_port}/{String_database}")
    print("Successfully connected to DB")
except exception as e:
    print(e)
finally:
    engine.dispose()
    print("sql connection is closed.")

Successfully connected to DB
sql connection is closed.


In [61]:
df.to_sql(name= 'olist_order_payments' , if_exists= "replace" , con= engine )

103886

In [62]:
df.memory_usage(deep=True).sum() / 1024**2  # in MB

np.float64(17.814587593078613)

In [63]:
query = "select * from olist_order_payments;"

sql_df = pd.read_sql(query , engine)

In [ ]:
sql_df.dtypes

index                     int64
order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object